In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


# StructuredOutputParser

In [3]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate

In [4]:
# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(name="source", description="사용자의 질문에 답하기 위해 사용된 '출저', '웹사이트주소' 이여야 합니다.")
]

# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [5]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력변수로 'question' 사용
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [6]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = prompt | model | output_parser

In [7]:
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '대한민국의 수도는 서울입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

In [8]:
for s in chain.stream({"question": "세종대황의 업적은 무엇인가요?"}):
    print(s)

{'answer': '세종대왕은 한글 창제, 과학 기술 발전, 농업 및 음악 발전 등 다양한 업적을 남겼습니다. 특히, 1443년에 훈민정음을 창제하여 한국어의 표기 체계를 혁신하였고, 이는 한국 문화의 기초가 되었습니다. 또한, 천문학과 의학 분야에서도 많은 발전을 이루었으며, 농사직설과 같은 농업 관련 서적을 통해 농민의 삶을 개선하고자 했습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}
